# WEB SCRAPING CON BEAUTIFUL SOUP

El web scraping es el proceso de extraer datos de sitios web, y Python proporciona bibliotecas potentes como Beautiful Soup 4 que facilitan el raspado y análisis de contenido HTML y XML.

Importaciones

In [ ]:
# Importamos BeautifulSoup para parsear HTML
from bs4 import BeautifulSoup
# Importamos requests para hacer peticiones HTTP
import requests
# Importamos pandas para manejar los datos en DataFrames
import pandas as pd

Estamos recopilando datos de periféricos de computadora del sitio "MD Computers" usando Beautiful Soup 4

Introduce el artículo del cual quieres recopilar información.

In [ ]:
# Solicitamos al usuario que introduzca el artículo que quiere buscar
search_item = input("Which item do you want to search? ")

Desde la URL dinámica del sitio web estamos formateando el resultado de búsqueda en la URL.

In [ ]:
# Construimos la URL de búsqueda insertando el término que introdujo el usuario
url = f"https://mdcomputers.in/index.php?submit_search=&route=product%2Fsearch&&search={search_item}"

In [ ]:
url

Parseando el contenido HTML de la página.

In [ ]:
# Obtenemos el contenido HTML de la página como texto
page_ = requests.get(url).text
# Parseamos el HTML con BeautifulSoup para poder navegar por él
doc = BeautifulSoup(page_, "html.parser")

Encontrando el número total de páginas de resultados filtrando las etiquetas HTML por nombre de clase

In [ ]:
# Buscamos el elemento que contiene el número total de páginas
#total_pages_x = doc.find(class_="col-sm-6 text-right")
# Encontramos el elemento con la clase que muestra la paginación
total_pages_x = doc.find(class_="col-md-6 text-md-end mb-2")

In [ ]:

#Extrae el número de páginas del texto, tomando el valor que está entre paréntesis antes del espacio.
total_pages_x.text.split("(")[1].split(" ")[0]

In [ ]:
# Convertimos el número de páginas a entero para poder usarlo en el bucle
total_pages = int(total_pages_x.text.split("(")[1].split(" ")[0])

In [ ]:
# Mostramos el total de páginas
total_pages

Total de páginas de resultados obtenidas para la búsqueda

In [ ]:
# Verificamos el número total de páginas
total_pages

Vamos a recopilar el nombre del artículo, su enlace y su precio (antiguo y nuevo) a continuación

Creando listas vacías para los datos

In [ ]:
# Inicializamos listas vacías para almacenar los datos de cada producto
link = []          # Para guardar los enlaces de los productos
item_name = []     # Para guardar los nombres de los productos
new_price = []     # Para guardar los precios actuales
old_price = []     # Para guardar los precios anteriores

Aquí está el código principal que extrae la información del código HTML

Desde esta URL, podemos buscar los productos que necesitamos, iterando a través de las páginas hasta el total de páginas de resultados que obtuvimos anteriormente.

In [ ]:
# Iteramos a través de todas las páginas de resultados
for page in range(1, total_pages + 1):
    # Construimos la URL para cada página específica
    url = f"https://mdcomputers.in/index.php?route=product/search&page={page}&search={search_item}"
    print(url)
    # Obtenemos el contenido HTML de la página actual
    page_ = requests.get(url).text
    # Parseamos el HTML con BeautifulSoup
    doc = BeautifulSoup(page_, "html.parser")
    
    # Encontramos todos los elementos div que contienen información de productos
    items = doc.find_all(class_ = "product-wrapper") #Encontrando el div principal para cada producto que contiene los datos
    print(items)
    # Iteramos a través de cada producto encontrado
    for item in items:
        
        # Extraemos el enlace del producto del atributo href dentro del tag h3
        link.append((item.find("h3")).find("a")['href'])
        # Extraemos el nombre del producto del texto dentro del tag h3
        item_name.append(((item.find("h3")).find("a")).text)
        # Extraemos el precio antiguo (tachado) eliminando el símbolo de moneda
        old_price.append(((item.find("span",class_ = "del")).find("span",class_ = "amount")).text[1:])
        # Extraemos el precio nuevo (actual) eliminando el símbolo de moneda
        new_price.append(((item.find("span",class_ = "ins")).find("span",class_ = "amount")).text[1:])
        #try:
            # En algunos productos no hay precio antiguo, por lo que se usa try para manejar el error
        #    old_price.append(((item.find("div",class_ = "price")).find("span",class_ = "price-old")).string[1:])    
        #except:
        #    # Para productos sin precio antiguo, se declara como null
        #    old_price.append(None)

In [ ]:
# Verificamos el número total de nombres de productos recopilados
len(item_name)

In [ ]:
# Verificamos el número total de enlaces recopilados
len(link)

In [ ]:
# Mostramos la lista de nombres ordenados alfabéticamente
sorted(item_name)

Convirtiendo las listas a un Diccionario y luego a un DataFrame.

In [ ]:
# Creamos un diccionario con todas las listas de datos recopilados
my_dict = {'item_name': item_name , 'link': link , 'new_price': new_price , 'old_price': old_price}

In [ ]:
# Verificamos la longitud de la lista de precios antiguos
len(old_price)

In [ ]:
# Verificamos que todas las listas tengan la misma longitud
for key in my_dict:
    print(len(my_dict[key]))

In [ ]:
# Convertimos el diccionario en un DataFrame de pandas para facilitar el manejo de datos
df = pd.DataFrame(my_dict)

Aquí está nuestro DataFrame con los datos requeridos

In [ ]:
# Mostramos el DataFrame completo con todos los productos y sus datos
df

Convirtiendo el DataFrame a un documento de Excel con el nombre del archivo como el artículo que buscamos

In [ ]:
# Exportamos el DataFrame a un archivo Excel con el nombre basado en el término de búsqueda
df.to_excel(f'{search_item}_list_mdcomputers.xlsx')

Finalmente hemos extraído los datos del sitio web (MD Computers) exitosamente usando BeautifulSoup4.